# Optimizer（优化算法）
## 一、基础优化思路——梯度下降
### 损失函数 目标函数 代价函数
- 目标函数=每个样本损失函数的算数平均值
### （1.全批量）梯度下降
- 方法：目标函数对参数求梯度(O(n))，用此梯度·learning rate，得参数减小值<p> 梯度下降的1个迭代周期对模型参数只迭代1次。
- 优点：样本量最大，目标函数最接近  ```度量函数of(真实规律函数-现在参数确定的函数) ```,梯度下降少走弯路，迭代次数最少
- 缺点：虽然减少了迭代次数，但每次迭代复杂度至少O(n),计算量过大，训练慢
### 2.随机梯度下降
- 方法：随机选取一个样本的损失函数，对参数求梯度，以此梯度下降<p>
虽然随机梯度下降和梯度下降在一个迭代周期里都处理了1,500个样本，但实验中随机梯度下降的一个迭代周期耗时更多。<p>这是因为随机梯度下降在一个迭代周期里做了更多次的自变量迭代，而且单样本的梯度计算难以有效利用矢量计算。
- 优点：每次迭代O(1),速度最快
- 缺点：样本量最小，损失函数距离 ```度量函数of(真实规律函数-现在参数确定的函数)```最远（随机误差干扰），梯度下降走的弯路最多，因而迭代次数最多
### 3.小批量随机梯度下降
- 方法：从样本中 ```随机```(“随机”) 选取 ```batch_size``` （“小批量”）个样本，以其损失函数平均函数对参数梯度为所用梯度<p>
通常，小批量随机梯度在每个迭代周期的耗时介于梯度下降和随机梯度下降的耗时之间。
- 特点：调和全批量/随机梯度下降两方法的优缺点
- batch_size需要人为选取，是重要的超参数
- 小批量随机梯度下降中每次迭代的计算开销为O(batch_size)。当批量大小为1时，该算法即为随机梯度下降；当批量大小等于训练数据样本数时，该算法即为梯度下降。
- 这正是Pytorch为什么要有Dataloader的原因
- 仍有缺欠：若lr一定，则多个自变量的下降幅度同一（均为lr*grad）,这导致某些自变量正常收敛的时候，另一些变量在最优点附近震荡乃至发散，或者收敛速率过慢
## 二、梯度自适应——动量法
### 0.背景：
- SGD中，若lr一定，则多个自变量的下降幅度同一（均为lr*grad）,这导致某些自变量正常收敛的时候，另一些变量在最优点附近震荡乃至发散，或者收敛速率过慢
- 为此提出动量法等优化算法
### 1.指数加权平均
- 定义 
- 可发现，各项系数和的极限为1，是以是一种加权平均
- 偏差修正：正因为系数和*极限*为1，故在项数较小时系数和小于1，因此每项系数都引入1/1-beta^t因子，使系数和永为1，这才是真正意义上的加权平均(实际应用较少，why?)<p>
事实上若不进行偏差修正，则项数较少时数值会较小
- 近似计算：由lim<n -> inf> (1-1/n)^n=1/e,故认为 (1-beta)(bata^(1/1-beta))极小可不计(但事实上0.34左右的下界难道不正说明不容忽略？)，因此指数加权平均进一步近似为最近1/(1-beta)项的平均，具有无记忆性
- 总结：(在不进行偏差修正时)极限情况下无穷多项的指数加权平均 —近似—> 项数较多时（有限）所有项的指数加权平均 —近似—> 项数较多时最近1/1-beta项的指数加权平均
### 2.实现
- 深度学习框架中的优化求解器早已构建了动量法，SGD就是动量法下梯度下降
- v_t=beta*v_t-1 + (1-beta)g_t
- wi_t=wi_t-1 - lr*v_t
- 本质上相当于 *wi_t-=(exp-weighed)sigma(g_0~t)*
### 3.数学
- 核心问题：上述背景中的两大问题，或1.从学习率角度，在偏导过大时调低这个方向的学习率，在偏导过小时调高这个方向的学习率；或2.从偏导角度，（各方向学习率相同的前提下）如果一直很小而且正负性一致，则让*用以计算的*偏导放大一点；如果出现震荡（那么震荡前后偏导一定反号），则在检测到震荡后降低一点学习率，不要再次震荡回原始的*另一极*（当然，要能在震荡前就预判到会震荡（比如偏导太大，自变量变化幅度一定很大，很容易震荡）而在震荡出现前就调小一点偏导，防患未然，甚至更好，不过动量法没有做到）。
- 采用第二策解决。用动量法算法能满足第二策：这样一来，如果某个方向出现震荡，则此方向偏导正负抵消，在震荡一次后下降得较慢；某个方向梯度一直很小，则逐渐积累变大，下降幅度逐渐加快
- 这个算法为什么是这样的：关键在于，要实现上述方案，必须将最近几步的偏导纳入考量（因此需用平均数），但不能将很久以前（因而无关于当下）因素全纳入考量（因此不是算术平均等等平均，而是能逐渐降低前项的权重的指数加权平均）。
## 三、学习率自适应——Ada系列
### 0.背景
- 采用动量法“核心问题”部分的第一策，即不同方向上的学习率根据具体偏导大小自适应（因此各方向学习率不同）
### 1.算法
- AdaGrad==>RMSProp;AdaDelata(注：指数加权平均都涉及偏差修正)
- 详情见书
## 四、集上述两策之大成——Adam算法
- 按动量法算V_t
- 按RMSProp算S_t
- wi_t-=lr*v_t/(sqrt(S_t)+epx(mini_value))
##
## 总结
SGD====>*若lr一定，则多个自变量的下降幅度同一（均为lr\*grad）,这导致某些自变量正常收敛的时候，另一些变量在最优点附近震荡乃至发散，或者收敛速率过慢*====>Ada系列;动量法=====>Adam算法
